# **MPI**

MPI широко используется в научных и инженерных вычислениях, где требуется высокая производительность и масштабируемость параллельных вычислений. С его помощью можно создавать параллельные программы для различных приложений, от численного моделирования и анализа данных до расчетов на суперкомпьютерах.


# **Основные понятия**

MPI (Message Passing Interface): MPI - это стандарт для обмена сообщениями между процессами в параллельных вычислениях. Он позволяет программистам создавать параллельные приложения, в которых несколько процессов могут обмениваться данными и координировать свою работу.

Коммуникаторы MPI определяют группу процессов, между которыми можно обмениваться сообщениями. Существуют встроенные коммуникаторы, такие как MPI.COMM_WORLD, который включает все процессы, запущенные в рамках одной программы.

Обмен сообщениями: MPI предоставляет функции для отправки и приема сообщений между процессами. Это основной механизм для обмена данными в параллельных вычислениях.

Collective Communication: Кроме отправки и приема индивидуальных сообщений, MPI также предоставляет коллективные операции, такие как MPI_Bcast, MPI_Reduce и MPI_Scatter, которые позволяют одному процессу отправлять данные всем или некоторым другим процессам.

Библиотека mpi4py: Для работы с MPI в Python часто используется библиотека mpi4py, которая предоставляет доступ к функциям MPI через интерфейс Python.

Установим mpi4py


In [ ]:
pip install mpi4py


Проверим установкучерез файл

In [ ]:
%%writefile test.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print("Hello! I'm process", rank, "out of", size)

MPI.Finalize()


Writing test.py


Запустим программу


In [ ]:
!mpiexec -n 4 python test.py



--------------------------------------------------------------------------
mpiexec has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpiexec as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------


**Пример: Параллельное выполнение простых задач**


В этом примере создается несколько процессов MPI, каждый из которых выполняет свою задачу. Главный процесс с рангом 0 выполняет определенную задачу, в то время как остальные процессы (рабочие) выполняют другие задачи. Такое параллельное выполнение позволяет ускорить выполнение программы, разделяя работу между несколькими процессами.

In [ ]:
from mpi4py import MPI

# Инициализация MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Пример параллельного выполнения простых задач
if rank == 0:
    print("Master process is running")
    # Задача для главного процесса
else:
    print(f"Worker process {rank} is running")
    # Задача для рабочих процессов



Master process is running


**Пример : Использования сборки данных с нескольких процессои**

Этот пример создает несколько процессов MPI, каждый из которых генерирует данные на основе своего ранга. Затем собранные данные с каждого процесса собираются вместе с помощью функции gather на процессе с рангом 0.

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = rank + 1
gathered_data = comm.gather(data, root=0)

if rank == 0:
    print(f"Process {rank} gathered data: {gathered_data}")


Process 0 gathered data: [1]


**Пример: Использование библиотеки NumPy для генерации данных**\
Этот пример демонстрирует использование функции `bcast` для распространения данных от процесса с рангом 0 ко всем остальным процессам в коммуникаторе MPI. Сначала процесс с рангом 0 генерирует массив данных при помощи библиотеки NumPy. Затем используется функция `bcast`, чтобы передать этот массив от процесса с рангом 0 ко всем остальным процессам. Если процесс имеет ранг 0, он отправляет данные, иначе он просто принимает переданные данные. Таким образом, все процессы получают одинаковые данные, сгенерированные процессом с рангом 0.


In [ ]:
from mpi4py import MPI
import numpy as np

# Инициализация MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Генерация данных с использованием библиотеки NumPy
if rank == 0:
    data = np.random.rand(10)
    print("Generated data:", data)

# Рассылка данных от процесса с рангом 0 ко всем остальным процессам
data = comm.bcast(data if rank == 0 else None, root=0)
print("Rank", rank, "received data:", data)


Generated data: [0.90032337 0.34666114 0.98828865 0.9728472  0.76280179 0.11558258
 0.74625381 0.92558524 0.83581736 0.03676646]
Rank 0 received data: [0.90032337 0.34666114 0.98828865 0.9728472  0.76280179 0.11558258
 0.74625381 0.92558524 0.83581736 0.03676646]


 **Пример: Использование функции scatter и библиотеки mpi4y**

Пример демонстрирует использование функции scatter в библиотеке mpi4py для распределения данных между процессами MPI. В начале программы инициализируется MPI коммуникатор и определяется ранг текущего процесса. Затем процесс с рангом 0 создает массив numpy из 10 элементов, заполненных нулями, который будет распределяться между процессами. Для этого массива создается список с одним элементом - самим массивом. Затем используется функция scatter, чтобы каждый процесс получил одну часть данных. Каждый процесс выводит полученную часть данных и свой ранг. Таким образом, функция scatter позволяет распределить массив данных между процессами MPI для параллельной обработки

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# Процесс с рангом 0 создает исходные данные
if rank == 0:
    data = np.zeros(10)  # Создаем массив numpy с 10 элементами
else:
    data = None

# Распределение данных между процессами с использованием функции scatter
chunk = comm.scatter([data], root=0)  # Передаем список с одним элементом - массивом data
print("Rank", rank, "received data chunk:", chunk)

Rank 0 received data chunk: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



### Упражнение 1: Использование трансляции

**Задание**: Напишите программу, где процесс 0 передаёт массив всем остальным процессам с использованием функции `bcast`.



In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = [1, 2, 3, 4, 5]
else:
    data = None

data = comm.bcast(data, root=0)
print("Process", rank, "received data:", data)

Process 0 received data: [1, 2, 3, 4, 5]


### Упражнение 2: Сбор данных

**Задание**: Каждый процесс генерирует случайное число. Используйте `gather` для сбора всех чисел в одном массиве в процессе 0.

In [ ]:
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# Генерация случайного числа
rand_num = np.random.rand()

# Собираем все числа в процессе 0
all_rand_nums = comm.gather(rand_num, root=0)

if rank == 0:
    print("Random numbers gathered at root:", all_rand_nums)

Random numbers gathered at root: [0.2882202681921189]


### Упражнение 3: Рассылка суммы

**Задание**: Процесс 0 собирает числа от всех процессов, вычисляет их сумму и рассылает эту сумму всем процессам.

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# Все процессы генерируют случайное число
rand_num = np.random.rand()
print("Process", rank, "generated number:", rand_num)

# Собираем все числа в процессе 0
all_rand_nums = comm.gather(rand_num, root=0)

if rank == 0:
    total_sum = sum(all_rand_nums)
    print("Total sum calculated at root:", total_sum)
else:
    total_sum = None

# Рассылаем сумму всем процессам
total_sum = comm.bcast(total_sum, root=0)
print("Process", rank, "received total sum:", total_sum)

Process 0 generated number: 0.4290660787289786
Total sum calculated at root: 0.4290660787289786
Process 0 received total sum: 0.4290660787289786



### Упражнение 4: Параллельное вычисление числа Пи методом Монте-Карло

**Задание**: Используйте метод Монте-Карло для оценки числа Пи. Каждый процесс генерирует случайные точки и определяет, сколько из них попадает внутрь круга. Используйте `reduce` для сбора и суммирования результатов, а затем вычислите и распечатайте приближенное значение числа Пи на процессе с рангом 0.


In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Количество точек для генерации в каждом процессе
points_per_process = 1000000

# Генерация случайных точек и подсчёт количества точек внутри круга
count = 0
for _ in range(points_per_process):
    x, y = np.random.rand(2)
    if x**2 + y**2 <= 1:
        count += 1

# Собираем все счетчики в root процессе
total_count = comm.reduce(count, op=MPI.SUM, root=0)

# Вычисляем Пи на процессе 0
if rank == 0:
    pi_estimate = (4 * total_count) / (size * points_per_process)
    print("Estimated Pi value:", pi_estimate)

Estimated Pi value: 3.139368


### Упражнение 5: Параллельное вычисление среднего

**Задание**: Напишите программу, где каждый процесс генерирует список случайных чисел. Используйте `reduce` для нахождения общей суммы всех чисел, а затем вычислите среднее значение на процессе с рангом 0.

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Каждый процесс генерирует список из 10 случайных чисел
local_data = np.random.rand(10)
print("Process", rank, "generated data:", local_data)

# Используем MPI reduce для суммирования всех списков в процессе 0
total_data = comm.reduce(np.sum(local_data), op=MPI.SUM, root=0)

if rank == 0:
    # Вычисляем среднее значение
    mean_value = total_data / (size * 10)
    print("Mean value calculated on root process:", mean_value)

Process 0 generated data: [0.27075899 0.05401882 0.31366267 0.09037693 0.97128617 0.78278633
 0.43195784 0.32104946 0.22368985 0.73551486]
Mean value calculated on root process: 0.41951019265611755
